# Preparação

In [12]:
! pip install pandas

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [1]:
# importar as bibliotecas necessárias
import pandas as pd
import sqlite3


## Extração das informações

In [2]:
# conectar o banco de dados
conn = sqlite3.connect("saude_suplementar.db")

In [3]:
# executar a importação dos dados
query = """
    SELECT * FROM visao_geral

    WHERE status_operadora = "ativa";
"""

df = pd.read_sql(query, conn)

df.head()

,id_operadora,nome_operadora,porte,regiao,status_operadora,benef_marco,benef_junho,jan,fev,mar,abr,mai,jun
0,515,Allianz Saude S/A,1 a 5.000 beneficiarios,Sudeste,ativa,276,243,1,1,0,1,0,0
1,582,Porto Seguro - Seguro Saude S/A,Acima de 500.000 beneficiarios,Sudeste,ativa,678071,722612,366,370,370,372,394,416
2,701,Unimed Seguros Saude S/A,Acima de 500.000 beneficiarios,Sudeste,ativa,890000,911115,405,452,490,507,497,489
3,884,Itauseg Saude S.A.,5.001 a 10.000 beneficiarios,Sudeste,ativa,7232,7157,4,4,5,1,1,5
4,5711,Bradesco Saude S.A.,Acima de 500.000 beneficiarios,Sudeste,ativa,3075027,3078556,2545,2476,2393,2538,2712,2461


# Análise exploratória

## Descrição dos dados

In [4]:
# remover a coluna status_operadora
df.drop('status_operadora', axis = 1, inplace = True)

In [5]:
# exibir os formatos de dados por coluna
print(f"formato de dados do dataframe: \n", df.dtypes)

formato de dados do dataframe: 
 id_operadora       int64
nome_operadora    object
porte             object
regiao            object
benef_marco        int64
benef_junho        int64
jan                int64
fev                int64
mar                int64
abr                int64
mai                int64
jun                int64
dtype: object


In [6]:
# identificação de valores nulos
print(f"valores nulos: \n", df.isnull().sum())

valores nulos: 
 id_operadora      0
nome_operadora    0
porte             0
regiao            0
benef_marco       0
benef_junho       0
jan               0
fev               0
mar               0
abr               0
mai               0
jun               0
dtype: int64


In [7]:
# número de valores únicos por coluna
print(f"valores únicos: \n", df.nunique())

valores únicos: 
 id_operadora      553
nome_operadora    546
porte               8
regiao              5
benef_marco       552
benef_junho       552
jan                99
fev               100
mar                96
abr               103
mai               103
jun               101
dtype: int64


In [8]:
# estatísticas dos dados
print(f"estatísticas: \n", df.describe())

estatísticas: 
         id_operadora   benef_marco   benef_junho          jan          fev  \
count     553.000000  5.530000e+02  5.530000e+02   553.000000   553.000000   
mean   357294.788427  9.305849e+04  9.392187e+04    51.896926    48.710669   
std     55133.773880  3.309126e+05  3.349469e+05   277.731654   253.584483   
min       515.000000  4.000000e+00  0.000000e+00     0.000000     0.000000   
25%    324523.000000  9.755000e+03  9.852000e+03     1.000000     0.000000   
50%    352179.000000  2.328100e+04  2.314200e+04     3.000000     3.000000   
75%    412015.000000  6.237100e+04  6.303900e+04    16.000000    15.000000   
max    424188.000000  4.306431e+06  4.378569e+06  3943.000000  3387.000000   

               mar          abr          mai          jun  
count   553.000000   553.000000   553.000000   553.000000  
mean     47.146474    46.737794    49.849910    45.491863  
std     233.577374   227.978764   244.008013   218.644640  
min       0.000000     0.000000     0.000

# Métricas

## Número de reclamações por beneficiário

In [9]:
# criar as colunas com os valores acumulados de reclamações por trimestre
df['reclamacoes_1_trim'] = df['jan'] + df['fev'] + df['mar']

df['reclamacoes_2_trim'] = df['abr'] + df['mai'] + df['jun']

df['total_reclamacoes'] = df['reclamacoes_1_trim'] + df['reclamacoes_2_trim']

df.head(2)

,id_operadora,nome_operadora,porte,regiao,benef_marco,benef_junho,jan,fev,mar,abr,mai,jun,reclamacoes_1_trim,reclamacoes_2_trim,total_reclamacoes
0,515,Allianz Saude S/A,1 a 5.000 beneficiarios,Sudeste,276,243,1,1,0,1,0,0,2,1,3
1,582,Porto Seguro - Seguro Saude S/A,Acima de 500.000 beneficiarios,Sudeste,678071,722612,366,370,370,372,394,416,1106,1182,2288


In [10]:
# calcular o número de reclamações por beneficiário, por trimestre
df['per_capita_1_trim'] = df['reclamacoes_1_trim'] / df ['benef_marco']

df['per_capita_2_trim'] = df['reclamacoes_2_trim'] / df ['benef_junho']

df['per_capita_total'] = df['total_reclamacoes'] / df ['benef_junho']

df['per_capita_variacao'] = df['per_capita_2_trim'] - df['per_capita_1_trim']

df.head(2)

,id_operadora,nome_operadora,porte,regiao,benef_marco,benef_junho,jan,fev,mar,abr,mai,jun,reclamacoes_1_trim,reclamacoes_2_trim,total_reclamacoes,per_capita_1_trim,per_capita_2_trim,per_capita_total,per_capita_variacao
0,515,Allianz Saude S/A,1 a 5.000 beneficiarios,Sudeste,276,243,1,1,0,1,0,0,2,1,3,0.007246,0.004115,0.012346,-0.003131
1,582,Porto Seguro - Seguro Saude S/A,Acima de 500.000 beneficiarios,Sudeste,678071,722612,366,370,370,372,394,416,1106,1182,2288,0.001631,0.001636,0.003166,0.000005


In [11]:
# restringir a consulta às operadoras com mais de 5000 beneficiários
df_filtrado = df[df['benef_junho'] > 5000]

# Resultados

**Importante:**
* Exceto quando especificado o contrário, os resultados abaixo se referem ao valor total de reclamações do primeiro semestre de 2025.
* Para minimizar vieses relacionados ao baixo número de beneficiários, os resultados abaixo foram obtidos desconsiderando-se as operadoras com menos de 5000 usuários.

## Top 5 operadoras com o maior número de beneficiários

In [12]:
df_filtrado[['nome_operadora', 'benef_junho']] \
  .sort_values(by='benef_junho', ascending=False) \
  .head(5)

,nome_operadora,benef_junho
359,Hapvida Assistencia Medica S.A.,4378569
322,Notre Dame Intermedica Saude S.A.,3234572
4,Bradesco Saude S.A.,3078556
146,Amil Assistencia Medica Internacional S.A.,2923027
5,Sul America Companhia De Seguro Saude,2217466


## Top 5 operadoras com o maior número absoluto de reclamações

In [13]:
df_filtrado[['nome_operadora', 'benef_junho', 'total_reclamacoes', 'per_capita_total']] \
  .sort_values(by='total_reclamacoes', ascending=False) \
  .head(5)

,nome_operadora,benef_junho,total_reclamacoes,per_capita_total
322,Notre Dame Intermedica Saude S.A.,3234572,17807,0.005505
4,Bradesco Saude S.A.,3078556,15125,0.004913
62,Unimed Do Est. Do Rj Federacao Est. Das Coo,372058,13573,0.036481
5,Sul America Companhia De Seguro Saude,2217466,11195,0.005049
146,Amil Assistencia Medica Internacional S.A.,2923027,10998,0.003763


In [14]:
df_filtrado[['nome_operadora', 'benef_junho', 'total_reclamacoes', 'per_capita_total']] \
  .sort_values(by='benef_junho', ascending=False) \
  .head(1)

,nome_operadora,benef_junho,total_reclamacoes,per_capita_total
359,Hapvida Assistencia Medica S.A.,4378569,9364,0.002139


Com exceção da terceira posição (Unimed Rio de Janeiro), as operadoras aqui listadas correspondem àquelas que opcupam a posição 2-5 no ranking de maiores operadoras em número de beneficiários. Esse resultado sugere que o número absoluto de reclamações é fortemente influenciado pelo tamanho da base de beneficiários, o que limita sua utilidade como métrica isolada para avaliação de desempenho regulatório.

Cabe destacar, ainda, que a Hapvida (operadora com o maior número total de beneficiários) ocupa o sexto lugar neste ranking, enfatizando o papel do porte da operadora no seu desempenho geral em reclamações.

## Top 5 operadoras com o maior número de reclamações por usuário

In [15]:
df_filtrado[['nome_operadora', 'total_reclamacoes', 'benef_junho','per_capita_total']] \
  .sort_values(by='per_capita_total', ascending=False) \
  .head(5)

,nome_operadora,total_reclamacoes,benef_junho,per_capita_total
109,Ativia Servicos De Saude S/A,371,7724,0.048032
70,Federacao Das Sociedades Cooperativas De Tr,417,10506,0.039692
62,Unimed Do Est. Do Rj Federacao Est. Das Coo,13573,372058,0.036481
537,Integra Assistencia Medica Sa,1558,79040,0.019712
455,Cemig Saude,944,56467,0.016718


As cinco operadoras com o maior número de reclamações por beneficiário não fazem parte do ranking de maiores operadoras do Brasil, e apenas uma delas (Unimed Rio de Janeiro) pode ser identificada por meio da avaliação dos números absolutos. 

Ainda que fatores como escala e estrutura operacional possam influenciar esse resultado, a magnitude dos valores per capita observados sugere a existência de problemas que justificam investigação regulatória direcionada, visando à identificação de possíveis falhas processuais e sistêmicas que estejam relacionadas ao desempenho observado.

## Top 5 operadoras com a maior variação no número de reclamações por usuário

Calculado como número de reclamações por usuário do segundo trimestre - primeiro trimestre.

### **Aumento** no número de reclamações por usuário

In [16]:
df_filtrado[['nome_operadora', 'total_reclamacoes', 'reclamacoes_1_trim', 'reclamacoes_2_trim', 'benef_junho','per_capita_variacao']] \
  .sort_values(by='per_capita_variacao', ascending=False) \
  .head(5)

,nome_operadora,total_reclamacoes,reclamacoes_1_trim,reclamacoes_2_trim,benef_junho,per_capita_variacao
546,Nova Saude Operadora Integrada De Saude Ltd,256,82,174,18324,0.004340
529,Gocare Planos De Saude Ltda,295,106,189,40424,0.002237
335,Unimed Oeste Do Para - Cooperativa De Traba,537,270,267,33942,0.002152
72,Associacao De Beneficencia E Filantropia Sa,1288,514,774,145744,0.001946
412,Associacao Assistencial De Saude Suplementa,376,170,206,31352,0.001668


Operadoras com menor base de beneficiários podem apresentar maior volatilidade nos indicadores per capita, o que exige monitoramento contínuo para distinguir tendências estruturais de variações pontuais. Sugere-se o monitoramento desta métrica como indicador de desempenho, com acompanhamento da veriação trimestre a trimestre durante um período mais longo, a fim de identificar a necessidade de intervenções pontuais em operadoras com alta variação que não façam parte daquelas com um grande número de reclamações por beneficiário.

### **Diminuição** no número de reclamações por usuário

In [17]:
df_filtrado[['nome_operadora', 'total_reclamacoes', 'reclamacoes_1_trim', 'reclamacoes_2_trim', 'benef_junho','per_capita_variacao']] \
  .sort_values(by='per_capita_variacao', ascending=True) \
  .head(5)

,nome_operadora,total_reclamacoes,reclamacoes_1_trim,reclamacoes_2_trim,benef_junho,per_capita_variacao
109,Ativia Servicos De Saude S/A,371,266,105,7724,-0.013307
455,Cemig Saude,944,780,164,56467,-0.010636
519,Atitude Saude Assistencia Medica Ltda,91,63,28,8858,-0.004556
348,Dona Saude Clinicas Ltda,138,62,76,12916,-0.003952
62,Unimed Do Est. Do Rj Federacao Est. Das Coo,13573,7649,5924,372058,-0.003328


Operadoras com menor base de beneficiários podem apresentar maior volatilidade nos indicadores per capita, o que exige monitoramento contínuo para distinguir tendências estruturais de variações pontuais. Sugere-se o monitoramento desta métrica como indicador de desempenho, com acompanhamento da veriação trimestre a trimestre durante um período mais longo, a fim de identificar a eficácia de intervenções regulatórias, em especial nas operadoras com alto número de reclamações por usuário.